In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="0, 1, 2, 3, 4, 5, 6, 7"
os.environ["NCCL_DEBUG"]="WARN"
#os.environ["NCCL_P2P_LEVEL"]="NODE"

In [2]:
import sys
sys.path.append("../../../")

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
import omegaconf
import numpy as np
import matplotlib.pyplot as plt
from ganime.data.experimental import ImageDataset, VideoDataset
from ganime.model.vqgan_clean.vqgan import VQGAN
from ganime.visualization.videos import display_videos
from ganime.visualization.images import display_images
import tensorflow as tf
from datetime import datetime
from tqdm import tqdm
import os
from pyprojroot.pyprojroot import here
#tf.get_logger().setLevel('ERROR')

In [5]:
for device in tf.config.list_physical_devices("GPU"):
    tf.config.experimental.set_memory_growth(device, True)

In [6]:
strategy = tf.distribute.MirroredStrategy(cross_device_ops=tf.distribute.HierarchicalCopyAllReduce())

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1', '/job:localhost/replica:0/task:0/device:GPU:2', '/job:localhost/replica:0/task:0/device:GPU:3', '/job:localhost/replica:0/task:0/device:GPU:4', '/job:localhost/replica:0/task:0/device:GPU:5', '/job:localhost/replica:0/task:0/device:GPU:6', '/job:localhost/replica:0/task:0/device:GPU:7')


2022-08-09 03:15:49.426347: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-08-09 03:15:52.261950: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 22302 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:01:00.0, compute capability: 8.6
2022-08-09 03:15:52.263259: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 22302 MB memory:  -> device: 1, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:25:00.0, compute capability: 8.6
2022-08-09 03:15:52.264261: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/

In [7]:
cfg = omegaconf.OmegaConf.load(here("configs/moving_mnist_image.yaml"))

In [8]:
num_workers = len(tf.config.list_physical_devices("GPU"))
batch_size = cfg["trainer"]["batch_size"] 
global_batch_size = batch_size * strategy.num_replicas_in_sync
n_epochs = cfg["trainer"]["n_epochs"] 
sample_batch_size = 8

In [9]:
#dataset_length = 34045 # KNY
#dataset_length = 310153 #KNY full
dataset_length = 20*10000 # MNIST
num_batch = dataset_length / batch_size

In [10]:
# Normalizing the images to [-1, 1]
def normalize(image):
    image = (tf.cast(image, tf.float32) / 127.5) - 1

    return image

In [11]:
def random_crop(image, img_height=64, img_width=128):
    cropped_image = tf.image.random_crop(image, size=[img_height, img_width, 3])

    return cropped_image

In [12]:
@tf.function()
def random_jitter(image):
    # Resizing to 72x142
    image = tf.image.resize(image, [72, 142], method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)

    # Random cropping back to 64x128
    image = random_crop(image)

    if tf.random.uniform(()) > 0.5:
        # Random mirroring
        image = tf.image.flip_left_right(image)

    return image

In [13]:
def preprocess(image):
    image = tf.reshape(image, (tf.shape(image)[0], tf.shape(image)[1], 3))
    #image = random_jitter(image)
    image = normalize(image)
    return image, image

In [14]:
dataset = ImageDataset("../../../data/mnist_tfrecords").load()
dataset = dataset.shuffle(dataset_length, reshuffle_each_iteration=True, seed=10).map(preprocess, num_parallel_calls=tf.data.AUTOTUNE)

In [15]:
train_size = int(dataset_length * 0.8)
validation_size = int(dataset_length * 0.1)
test_size = int(dataset_length * 0.1)

In [16]:
train_ds = dataset.take(train_size)
validation_ds = dataset.skip(train_size).take(validation_size)#.padded_batch(global_batch_size).map(postprocess)
test_ds = dataset.skip(train_size + validation_size).take(validation_size)#.padded_batch(global_batch_size).map(postprocess)

In [17]:
train_sample_data = next(train_ds
                          .batch(sample_batch_size)
                          .prefetch(tf.data.AUTOTUNE).as_numpy_iterator())
validation_sample_data = next(validation_ds.batch(sample_batch_size).as_numpy_iterator())

In [18]:
train_ds = (train_ds.batch(global_batch_size, drop_remainder=True)
            .prefetch(tf.data.AUTOTUNE))
validation_ds = (validation_ds.batch(global_batch_size, drop_remainder=True)
            .prefetch(tf.data.AUTOTUNE))
test_ds = (test_ds.batch(global_batch_size, drop_remainder=True)
            .prefetch(tf.data.AUTOTUNE))

In [19]:
from ganime.utils.callbacks import TensorboardImage, get_logdir

logdir = "../../../logs/ganime/vqgan/moving_mnist_image_2022-08-08_05-05-23" #get_logdir("../../../logs/ganime/vqgan", experiment_name="moving_mnist_image")
# Define the basic TensorBoard callback.
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)
tensorboard_image_callback = TensorboardImage(logdir, train_sample_data, validation_sample_data)
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor="val_total_loss",
    min_delta=0.0001,
    patience=100,
    restore_best_weights=True,
)
checkpointing = tf.keras.callbacks.ModelCheckpoint(os.path.join(logdir, "checkpoint", "checkpoint"), monitor='val_reconstruction_loss', save_best_only=True, save_weights_only=True)
callbacks = [tensorboard_callback, tensorboard_image_callback, checkpointing]

In [20]:
# train_mean, train_var, train_std = dataset_statistics(train_ds)

In [21]:
from ganime.visualization.images import display_true_pred

In [22]:
with strategy.scope():
    vqgan = VQGAN(**cfg["model"], num_replicas=strategy.num_replicas_in_sync)

Working with z of shape (1, 512, 16, 16) = 131072 dimensions.
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:lo

In [23]:
with strategy.scope():
    gen_optimizer = tf.keras.optimizers.Adam(
                learning_rate=cfg["trainer"]["gen_lr"],
                beta_1=cfg["trainer"]["gen_beta_1"],
                beta_2=cfg["trainer"]["gen_beta_2"],
                clipnorm=cfg["trainer"]["gen_clip_norm"],
    )
    disc_optimizer = tf.keras.optimizers.Adam(
        learning_rate=cfg["trainer"]["disc_lr"],
        beta_1=cfg["trainer"]["disc_beta_1"],
        beta_2=cfg["trainer"]["disc_beta_2"],
        clipnorm=cfg["trainer"]["disc_clip_norm"],
    )
    vqgan.compile(gen_optimizer=gen_optimizer, disc_optimizer=disc_optimizer)

In [ ]:
history = vqgan.fit(train_ds, validation_data=validation_ds, initial_epoch=211, epochs=n_epochs, callbacks=callbacks)

Epoch 212/10000
INFO:tensorflow:batch_all_reduce: 217 all-reduces with algorithm = hierarchical_copy, num_packs = 1
INFO:tensorflow:batch_all_reduce: 13 all-reduces with algorithm = hierarchical_copy, num_packs = 1
INFO:tensorflow:batch_all_reduce: 217 all-reduces with algorithm = hierarchical_copy, num_packs = 1
INFO:tensorflow:batch_all_reduce: 13 all-reduces with algorithm = hierarchical_copy, num_packs = 1


2022-08-09 03:23:09.940293: I tensorflow/stream_executor/cuda/cuda_dnn.cc:366] Loaded cuDNN version 8100
2022-08-09 03:23:10.734442: I tensorflow/stream_executor/cuda/cuda_dnn.cc:366] Loaded cuDNN version 8100
2022-08-09 03:23:11.635823: I tensorflow/stream_executor/cuda/cuda_dnn.cc:366] Loaded cuDNN version 8100
2022-08-09 03:23:13.082970: I tensorflow/stream_executor/cuda/cuda_dnn.cc:366] Loaded cuDNN version 8100
2022-08-09 03:23:14.448965: I tensorflow/stream_executor/cuda/cuda_blas.cc:1774] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2022-08-09 03:23:14.519365: I tensorflow/stream_executor/cuda/cuda_dnn.cc:366] Loaded cuDNN version 8100
2022-08-09 03:23:15.901031: I tensorflow/stream_executor/cuda/cuda_dnn.cc:366] Loaded cuDNN version 8100
2022-08-09 03:23:17.090509: I tensorflow/stream_executor/cuda/cuda_dnn.cc:366] Loaded cuDNN version 8100
2022-08-09 03:23:18.228551: I tensorflow/stream_executor/cuda/cuda_dnn.cc:366] Loaded cuDNN ve

625/625 [==============================] - 908s 732ms/step - total_loss: 6.7460 - reconstruction_loss: 0.2627 - vq_loss: 0.7530 - disc_loss: 0.0000e+00 - val_total_loss: 3.8047 - val_reconstruction_loss: 0.2821 - val_vq_loss: 0.7140 - val_disc_loss: 0.0000e+00
Epoch 213/10000
625/625 [==============================] - 354s 556ms/step - total_loss: 7.9167 - reconstruction_loss: 0.3200 - vq_loss: 0.7203 - disc_loss: 0.0000e+00 - val_total_loss: 3.2895 - val_reconstruction_loss: 0.3448 - val_vq_loss: 0.6930 - val_disc_loss: 0.0000e+00
Epoch 214/10000
625/625 [==============================] - 349s 552ms/step - total_loss: 8.4106 - reconstruction_loss: 0.3914 - vq_loss: 0.6878 - disc_loss: 0.0000e+00 - val_total_loss: 3.6512 - val_reconstruction_loss: 0.4143 - val_vq_loss: 0.7111 - val_disc_loss: 0.0000e+00
Epoch 215/10000
625/625 [==============================] - 351s 553ms/step - total_loss: 7.8104 - reconstruction_loss: 0.4386 - vq_loss: 0.6736 - disc_loss: 0.0000e+00 - val_total_loss:

In [ ]:
with strategy.scope():
    x = train_sample_data[0]
    generated = vqgan(x[:10])[0]

In [ ]:
quant, indices, _ = vqgan.encode(x[:10])

In [ ]:
with strategy.scope():
    quant = vqgan.quantize.get_codebook_entry(
            [list(range(0, 128))], shape=(1, 8, 16, 128)
    )

    decoded = vqgan.decode(quant)

In [ ]:
from ganime.model.vqgan_clean.discriminator.model import NLayerDiscriminator
with strategy.scope():
    discriminator = NLayerDiscriminator()


In [ ]:
with strategy.scope():
    vqgan.discriminator = discriminator

In [ ]:
model_input = tf.keras.layers.Input((256, 512, 6))
model = tf.keras.Model(inputs=[model_input], outputs=vqgan.discriminator.call(model_input))
model.summary()

In [ ]:
disc_out = vqgan.discriminator(tf.concat([tf.ones((8, 256, 512, 3)), tf.image.resize(
            train_sample_data[0], [256, 512], method=tf.image.ResizeMethod.NEAREST_NEIGHBOR
        )], axis=-1), training=False)

In [ ]:
plt.imshow(disc_out[0, ..., -1], vmin=-20, vmax=20, cmap='RdBu_r')
plt.colorbar()

In [ ]:
lengths = [789, 799, 814, 816 , 819 ,825 ,794 ,766 ,760 ,747,752 ,787 ,742 ,795 ,752 ,774 ,789, 
 770 ,786 ,770 ,813 ,805 ,802 ,787 ,744 ,800 ,890 ,732 ,830 ,762 ,667 ,751, 821 , 1541 ,
 784 ,801 ,788 ,768 ,768 ,787 ,726 ,705 ,647, 1102]

In [ ]:
len(lengths)

In [ ]:
sum(lengths)